In [35]:
import pandas as pd
import glob
import os
import random
import torch
import torch.nn as nn

# Corrected the path and pattern to match .parquet files
path = r'/Users/abhinavrajput/Desktop/PROJECTS/NoEEG_Borealis/doa-zero-eeg-sample_filtered'
all_files = glob.glob(os.path.join(path, "*.parquet"))

li = []

for filename in all_files:
    # Removed the index_col and header parameters
    df = pd.read_parquet(filename)
    li.append(df)

random.shuffle(li)

#frame = pd.concat(li, axis=0, ignore_index=True)


In [36]:
import numpy as np
from scipy.interpolate import interp1d

data = []
data.append(li[0:20])
data.append(li[20:40])
data.append(li[40:60])
data.append(li[60:80])
data.append(li[80:100])

# def noise_interpolate(y, scale, noise_probability=0.25):
#     x_valid = np.where(~np.isnan(y))[0]

#     # Check if there are enough points for quadratic interpolation
#     if len(x_valid) >= 3:
#         # Fit a quadratic spline on the valid data points
#         y_valid = y[x_valid]
#         quadratic_spline = interp1d(x_valid, y_valid, kind='quadratic', fill_value="extrapolate")
#         # Interpolate the missing (NaN) values using the spline
#         y_filled = quadratic_spline(np.arange(len(y)))
#     else:
#         # Convert the NumPy array to a pandas Series for easy handling of forward and backward fill
#         series = pd.Series(y)
#         # Apply forward fill and then backward fill
#         series.fillna(method='ffill', inplace=True)
#         series.fillna(method='bfill', inplace=True)
#         y_filled = series.to_numpy()

#     # Calculate noise scale based on the data range
#     noise_scale = scale * (np.max(y_filled) - np.min(y_filled))

#     # Decide where to add noise based on the probability
#     noise_mask = np.random.rand(*y_filled.shape) < noise_probability

#     # Generate Gaussian noise
#     noise = np.random.normal(loc=0, scale=noise_scale, size=y_filled.shape)

#     # Apply noise only where the mask is True
#     y_noisy = y_filled + (noise * noise_mask)

#     return y_filled, y_noisy


def noise_interpolate(y, scale, noise_probability = 0.25):
    x_valid = np.where(~np.isnan(y))[0]
    y_valid = y[x_valid]  # Correctly extract y_valid from y using indices
    x_missing = np.where(np.isnan(y))[0]

    # Fit a quadratic spline on the valid data points
    quadratic_spline = interp1d(x_valid, y_valid, kind='quadratic', fill_value="extrapolate")

    # Interpolate the missing (NaN) values using the spline
    y[x_missing] = quadratic_spline(x_missing)

    noise_scale = scale*(np.max(y) - np.min(y))

    noise_mask = np.random.rand(*y.shape) < noise_probability


    # Generate Gaussian noise
    noise = np.random.normal(loc=0, scale=noise_scale, size=y.shape)*noise_mask

    # Add noise to the polynomial values
    y_noisy = y + noise

    return y,y_noisy


def fill_nan(array):
    # Forward fill
    for i in range(1, len(array)):
        if np.isnan(array[i]):
            array[i] = array[i-1]

    # Backward fill
    for i in range(len(array)-2, -1, -1):
        if np.isnan(array[i]):
            array[i] = array[i+1]

    return array



In [37]:
import copy

def generate_df(data):
    #_,pnim= noise_interpolate(data['PNIm'].values.flatten(),0.02 )

    pnim = fill_nan(data['PNIm'])

    bis_ = data[['BIS']].values

    i = 0
    while(not (bis_[i] >= 0)):
        i = i+1

    dataf = copy.deepcopy(data)
    dataf.fillna(method='ffill', inplace=True)


    bis = data[['BIS']].values
    fc = dataf[['FC']].values
    spo2 = dataf[['SpO₂']].values
    co2 = dataf[['CO₂fe']].values

    bis = bis[i:].flatten()
    fc = fc[i:].flatten()
    spo2 = spo2[i:].flatten()
    co2 = co2[i:].flatten()
    pnim = pnim[i:]#.flatten()
    

    stacked_array = np.vstack([pnim,fc,spo2,co2,bis]).T
    df = pd.DataFrame(stacked_array, columns=['pnim', 'fc', 'spo2', 'co2', 'bis'])
    return df



In [38]:
######## K FOLD TRAINING #######

def kfold(lst, k):
    n = len(lst)
    slice_size = n // k  
    remainder = n % k  
    slices = []
    start = 0
    for i in range(k):
        end = start + slice_size + (1 if remainder > 0 else 0)
        remainder -= 1
        slices.append(lst[start:end])
        start = end
    return slices


In [39]:
def create_sequences(B, C, D, E, A, seq_length):
    sequences = []
    targets = []
    for i in range(len(A) - seq_length):
        # Stack sequences of B, C, D, E vertically, then transpose
        seq = np.vstack((B[i:i+seq_length], C[i:i+seq_length], D[i:i+seq_length], E[i:i+seq_length])).T
        target = A[i+seq_length]  # The target value to predict
        sequences.append(seq)  # Add the sequence to the list
        targets.append(target)  # Add the target to the list
    return torch.FloatTensor(sequences), torch.FloatTensor(targets)

In [40]:
i = 0
list = []
while i < 100:
    print(i)
    dat = li[i]
    df = generate_df(dat)
    list.append(df)
    i = i+1


fin_df = pd.concat(list, axis=0, ignore_index=True)




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [41]:
dff = fin_df.dropna(subset=['bis'])
dff.fillna(method='ffill', inplace=True)
nan_count = dff.isna().sum().sum()
print("Number of NaN values:", nan_count)


dff.to_csv('/Users/abhinavrajput/Desktop/PROJECTS/NoEEG_Borealis/ETDataset-main/final_surgeries.csv', index=False)


/var/folders/km/56dj4cx57fb70xxvgbb9g_500000gn/T/ipykernel_67864/1043079411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff.fillna(method='ffill', inplace=True)


Number of NaN values: 0
